In [ ]:
import os
import time

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.backends.cudnn as cudnn
import torch.nn.functional as F

import data_utils
import model as model

import random
import numpy as np

from sklearn.preprocessing import OneHotEncoder

In [ ]:
# Load data

train_dict = np.load('../data/training_dict.npy', allow_pickle=True).item()
valid_dict = np.load('../data/validation_dict.npy', allow_pickle=True).item()
test_dict = np.load('../data/testing_dict.npy', allow_pickle=True).item()
category_features = np.load(os.path.join('../data/category_feature.npy'), allow_pickle=True).item()
visual_features = np.load(os.path.join('../data/visual_feature.npy'), allow_pickle=True).item()

# Get the number of users and items

user_num = max(max(train_dict), max(valid_dict, default=-1), max(test_dict, default=-1)) + 1

item_num = max(
    max((max(items, default=-1) for items in train_dict.values()), default=-1),
    max((max(items, default=-1) for items in valid_dict.values()), default=-1),
    max((max(items, default=-1) for items in test_dict.values()), default=-1)
) + 1

print('Number of users: %d, Number of items: %d' % (user_num, item_num))

# Prepare training, validation, and test data

train_data = [[int(user), int(item)] for user, items in train_dict.items() for item in items]
valid_gt = [[int(user), int(item)] for user, items in valid_dict.items() for item in items]
test_gt = [[int(user), int(item)] for user, items in test_dict.items() for item in items]
print('Training samples: %d, Validation samples: %d, Test samples: %d' % (len(train_data), len(valid_gt), len(test_gt)))

# Load item features

category_feature_size = len(category_features)
unique_categories = set(category_features.values())
category_encoder = OneHotEncoder()
category_encoder.fit(np.array(list(unique_categories)).reshape(-1, 1))
category_features_onehot = category_encoder.transform(np.array(list(category_features.values())).reshape(-1, 1)).toarray()
print('Category features shape: %s' % str(category_features_onehot.shape))

visual_feature_size = len(visual_features)
example_key = next(iter(visual_features.keys()))
print('Visual features shape: %s' % str(visual_features[example_key].shape))

# Create user profiles

def create_user_profiles(interaction_dict, category_features, visual_features):

    user_profiles = {
        user_id: {
            'category_sum': np.zeros(category_features_onehot.shape[1]), 
            'visual_sum': np.zeros(visual_features[0].shape),
            'count': 0
        }
        for user_id in interaction_dict
    }

    for user_id, items in interaction_dict.items():
        for item_id in items:
            user_profiles[user_id]['category_sum'] += category_features[item_id]
            user_profiles[user_id]['visual_sum'] += visual_features[item_id]
            user_profiles[user_id]['count'] += 1

    # Averaging the features for each user profile
    for profile in user_profiles.values():
        if profile['count'] > 0:
            profile['category_sum'] /= profile['count']
            profile['visual_sum'] /= profile['count']

    return user_profiles

train_user_profiles = create_user_profiles(train_dict, category_features_onehot, visual_features)
valid_user_profiles = create_user_profiles(valid_dict, category_features_onehot, visual_features)
test_user_profiles = create_user_profiles(test_dict, category_features_onehot, visual_features)

In [ ]:
class ContentBasedModel(nn.Module):
    def __init__(self, num_categories, num_visual_features, hidden_dim):
        super(ContentBasedModel, self).__init__()
        # User category pathway
        self.user_category_fc = nn.Linear(num_categories, hidden_dim)
        
        # Item category pathway
        self.item_category_fc = nn.Linear(num_categories, hidden_dim)
        
        # User visual pathway
        self.user_visual_fc = nn.Linear(num_visual_features, hidden_dim)
        
        # Item visual pathway
        self.item_visual_fc = nn.Linear(num_visual_features, hidden_dim)
        
        # Combined features for prediction
        self.combined_fc = nn.Linear(hidden_dim * 4, hidden_dim)  # *4 because we concatenate user+item category+visual features
        self.output_layer = nn.Linear(hidden_dim, 1)

    def forward(self, user_category, user_visual, item_category, item_visual):
        # Process features through respective pathways
        user_category_out = F.relu(self.user_category_fc(user_category))
        item_category_out = F.relu(self.item_category_fc(item_category))
        user_visual_out = F.relu(self.user_visual_fc(user_visual))
        item_visual_out = F.relu(self.item_visual_fc(item_visual))
        
        # Combine all pathways
        combined_features = torch.cat((user_category_out, item_category_out, user_visual_out, item_visual_out), dim=1)
        
        # Further processing for final prediction
        combined_out = F.relu(self.combined_fc(combined_features))
        output = torch.sigmoid(self.output_layer(combined_out))
        return output

In [ ]:
# Initialise parameters

seed = 4242
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.set_num_threads(torch.get_num_threads())

data_path = "../data/"
model = "MF"
emb_size = 100
lr = 0.001
dropout = 0.0
batch_size = 100
epochs = 10
device = "cpu"
top_k = [10, 20, 50, 100]
log_name = "log"
model_path = "./models/"
num_categories = 368
num_visual_features = 512
embedding_dim = 32
hidden_dim = 32

In [ ]:
# Prepare the training data
train_dataset = data_utils.CBFData(
    user_item_pairs=train_data, 
    num_items=item_num, 
    category_features=category_features_onehot,
    visual_features=visual_features,
    user_profiles=train_user_profiles, 
    train_dict=train_dict, 
    is_training=True
)

train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)


In [ ]:
# Evaluation function

def evaluate_CBF(model, user_profiles, category_features, visual_features, gt_dict, train_dict, device, top_k=(10, 20, 50, 100)):
    model.eval()
    recalls = []
    ndcgs = []

    # No need to prepare these tensors in advance since we'll process each item individually
    with torch.no_grad():
        for K in top_k:
            recall_sum = 0.0
            ndcg_sum = 0.0
            num_users = len(gt_dict)

            for user_id, true_items in gt_dict.items():
                scores = np.zeros(len(category_features))

                user_category = torch.tensor(user_profiles[user_id]['category_sum'], dtype=torch.float32).unsqueeze(0).to(device)
                user_visual = torch.tensor(user_profiles[user_id]['visual_sum'], dtype=torch.float32).unsqueeze(0).to(device)
                
                for item_id in range(len(category_features)):
                    # Skip items the user has seen in training
                    if item_id in train_dict.get(user_id, []):
                        continue

                    item_category = torch.tensor(category_features[item_id], dtype=torch.float32).unsqueeze(0).to(device)
                    item_visual = torch.tensor(visual_features[item_id], dtype=torch.float32).unsqueeze(0).to(device)

                    # Score the user-item pair
                    score = model(user_category, user_visual, item_category, item_visual).squeeze().cpu().numpy()
                    scores[item_id] = score
                
                # Exclude scores for training items by setting them to -inf
                scores[list(train_dict.get(user_id, []))] = -np.inf

                # Get top-K items based on scores
                top_k_items = np.argsort(scores)[-K:]
                
                # Calculate metrics
                num_hits = len(set(top_k_items) & set(true_items))
                recall = num_hits / float(len(true_items))
                recall_sum += recall

                ndcg = calculate_ndcg(top_k_items, true_items, K)
                ndcg_sum += ndcg
            
            recalls.append(recall_sum / num_users)
            ndcgs.append(ndcg_sum / num_users)

    return recalls, ndcgs

def calculate_ndcg(predicted_items, true_items, K):
    dcg = 0.0
    idcg = 0.0
    for i, pred in enumerate(predicted_items[-K:]):
        if pred in true_items:
            dcg += 1.0 / np.log2(i + 2)
    for i in range(min(len(true_items), K)):
        idcg += 1.0 / np.log2(i + 2)
    return dcg / idcg if idcg > 0 else 0.0

In [ ]:

# Training the model

model = ContentBasedModel(num_categories, num_visual_features, hidden_dim)

# Define the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCELoss()

best_recall = 0

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        user_category, user_visual, item_category, item_visual, labels = batch
        user_category, user_visual = user_category.to(device), user_visual.to(device)
        item_category, item_visual = item_category.to(device), item_visual.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        predictions = model(user_category, user_visual, item_category, item_visual).squeeze()
        loss = criterion(predictions, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")

    # Evaluation
    recalls, ndcgs = evaluate_CBF(model, valid_user_profiles, category_features_onehot, visual_features, valid_dict, train_dict, device, top_k=[10, 20, 50, 100])
    print(f"[Validation] Recall: {recalls}, NDCG: {ndcgs}")

    # Update best recall and save model if necessary
    current_best_recall = max(recalls)
    if current_best_recall > best_recall:
        best_recall = current_best_recall
        # Save the model checkpoint
        torch.save(model.state_dict(), 'best_model.pth')
        print(f"New best model saved with recall: {best_recall}")

print("Training completed.")
# Test the model

In [ ]:
# Testing the model

model = ContentBasedModel(num_categories, num_visual_features, hidden_dim)
model.load_state_dict(torch.load("./best_model.pth"))
model.to(device)

model.eval()
recalls, ndcgs = evaluate_CBF(model, test_user_profiles, category_features_onehot, visual_features, test_dict, train_dict, device, top_k=[10, 20, 50, 100])
print(f"[Test] Recall: {recalls}, NDCG: {ndcgs}")